In [1]:
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
import tensorflow_addons as tfa
import sys
import os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))


import automatic_speech_recognition as asr
import time
from datetime import datetime
import argparse
import pickle

In [2]:
import numpy as np
import pandas as pd


In [3]:
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [4]:
from automatic_speech_recognition.model.quartznet import QUARTZNET_LAYERS

In [5]:
import horovod.tensorflow.keras as hvd

In [6]:
#%load_ext tensorboard
#%tensorboard --logdir=./models/ --port=32779

In [7]:
%pdb off

Automatic pdb calling has been turned OFF


## Train

In [8]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [9]:
# Initialize Horovod
hvd.init()
# Pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

In [10]:
def get_pipeline(model, optimizer=None):
    alphabet = asr.text.Alphabet(lang='en')
    features_extractor = asr.features.FilterBanks(
        features_num=64,
        standardize=None,
        winlen=0.02,
        winstep=0.01,
    )
    if not optimizer:
        optimizer = tf.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
    decoder = asr.decoder.GreedyDecoder()
    pipeline = asr.pipeline.CTCPipeline(
        alphabet, features_extractor, model, optimizer, decoder
    )
    return pipeline

In [11]:
def train_model(filename, dataset_idx, val_dataset_idx=None, initial_lr=0.001, batch_size=10, epochs=25, tensorboard=False, restart_filename=None,
               is_mixed_precision=False, n_blocks=1):
    basename = os.path.basename(filename).split('.')[0]
    model_dir = os.path.join(os.path.dirname(filename), basename + '_train')
    os.makedirs(model_dir, exist_ok=True)
    
    model = asr.model.get_quartznet(64, 29, is_mixed_precision=is_mixed_precision, num_b_block_repeats=n_blocks)

    if restart_filename:
        model.load_weights(restart_filename)
    
    batch_size_global = batch_size * hvd.size()
    initial_lr_global = initial_lr * hvd.size()
    
    dataset = asr.dataset.Audio.from_csv(dataset_idx, batch_size=batch_size_global, use_filesizes=True)
    dataset.sort_by_length()
    dataset.shuffle_indices()
    if val_dataset_idx:
        val_dataset = asr.dataset.Audio.from_csv(val_dataset_idx, batch_size=batch_size_global, use_filesizes=True)

    opt_instance = tf.optimizers.Adam(initial_lr_global, beta_1=0.9, beta_2=0.999)
    #opt_instance = tfa.optimizers.NovoGrad(initial_lr_global, beta_1=0.95, beta_2=0.5, weight_decay=0.001)

    opt = hvd.DistributedOptimizer(opt_instance)
    pipeline = get_pipeline(model, opt)
    
    callbacks = [
        hvd.callbacks.BroadcastGlobalVariablesCallback(0),
        hvd.callbacks.MetricAverageCallback(),
    ]
    schedule=tf.keras.experimental.CosineDecayRestarts(
        initial_lr_global, 10, t_mul=2.0, m_mul=1.0, alpha=0.0,
    )
    callbacks.append(LearningRateScheduler(schedule))
    if hvd.rank() == 0:
        prefix = datetime.now().strftime("%Y%m%d-%H%M%S")
        monitor_metric_name = 'loss' # if not val_dataset_idx else 'val_loss'  # val_loss is wrong and broken
        callbacks.append(
            keras.callbacks.ModelCheckpoint(
                os.path.join(model_dir, prefix + '_best.h5'),
                monitor=monitor_metric_name, save_weights_only=True,
                save_best_only=True))
        if tensorboard:
            logdir = os.path.join(model_dir, 'tb', prefix)
            tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
            callbacks.append(tensorboard_callback)

    time_start = time.time()

    hist = pipeline.fit(dataset, epochs=epochs, dev_dataset=val_dataset,
                        #steps_per_epoch=270,
                        callbacks=callbacks,
                        verbose=1 if hvd.rank() == 0 else 0)
    elapsed = time.time() - time_start
    
    if hvd.rank() == 0:
        print(f'Elapsed time: {elapsed}')
        #np.save(os.path.join(model_dir, prefix + '_hist.p'), np.array(hist))

In [12]:
#train_model(
#    filename='./models/quartznet_5x5_mp.h5', 
#    dataset_idx='./data/train-clean-100-index.csv',
#    val_dataset_idx='./data/dev-clean-index.csv',
#    is_mixed_precision=True,
#    batch_size=32,
#    initial_lr=0.005,
#    epochs=100,
#    tensorboard=False,
#    restart_filename=None,
#)

In [16]:
dataset = asr.dataset.Audio.from_csv('./data/train-clean-100-index.csv', batch_size=128, use_filesizes=True)
extractor = asr.features.FilterBanks(features_num=64, winlen=0.02, winstep=0.01)

In [24]:
ft, ln = extractor(dataset.get_batch(0)[0])

In [27]:
np.array(ft).size

13721600

'rnn_72' => 'rnn_216' [13], 'time_distributed_144' => 'time_distributed_431' [10], 'time_distributed_145' => 'time_distributed_432' [15]

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description='Train ctc asr model')
    parser.add_argument('--filename', type=str,
                        help='filename of the model')
    parser.add_argument('--dataset', type=str,
                        help='path to the dataset index',)
    parser.add_argument('--val_dataset', type=str,
                        help='path to the validation dataset index (optional)',
                        default=None)
    parser.add_argument('--batch_size', type=int,
                       help='batch size for training and validation',
                       default=10)
    parser.add_argument('--lr', type=float,
                       help='initial learning rate',
                       default=0.005)
    parser.add_argument('--epochs', type=int,
                       help='number of epochs to use for training',
                       default=25)
    parser.add_argument('--tensorboard', type=bool,
                       help='if tensorboard log will be written',
                       default=False)
    parser.add_argument('--restart_filename', type=str,
                       help='filename of the checkpoint to restart from',
                       default=None)
    parser.add_argument('--mp', type=bool,
                       help='if mixed precision training is requested',
                       default=False)
    args = parser.parse_args()
    
    train_model(filename=args.filename, dataset_idx=args.dataset,
                val_dataset_idx=args.val_dataset, epochs=args.epochs,
                tensorboard=args.tensorboard, restart_filename=args.restart_filename,
                is_mixed_precision=args.mp, initial_lr=args.lr)

training times: 25 epochs - 10 batch / epoch
 tt mixed - gpu - 9427s - batch = 1
 tt mixed - cpu - 29900s - batch = 1
 tt full  - cpu - 14050s - batch = 1